# 🔥 Fire...or No Fire?

## **Step 1:** Setup
We need to set up the dataframe to initial state.
Then we can manipulate it more in later steps, by cleaning it, digesting it, etc.

### Import Packages
> "If I have seen further, it is by standing on the shoulders of Giants."<br />
> &mdash; Isaac Newton


In [78]:
# glob crawler for reading files and folders
import glob

# regex parser for advanced string-parsing
import re

# pandas used for dataframe manipulation
import pandas

# numpy is used for odds and ends, like high-efficiency arrays
import numpy

# python imaging library; used for deconstructing images
from PIL import Image

# other imports are described as they are used
from keras.models import Sequential
from keras.losses import binary_crossentropy
from keras.optimizers import Adadelta
from keras.preprocessing.image import img_to_array
from keras.tensorflow_backend import _get_available_gpus
from sklearn.model_selection import train_test_split

ModuleNotFoundError: No module named 'keras.tensorflow_backend'

In [ ]:
_get_available_gpus()

### Crawl Datafiles
The **glob** package is great and sets up our project for success.
Be sure you have your directory-structure set up correctly as stated in the README&hellip;
Otherwise, I cannot guarentee that this project will work for you!

In [61]:
# get all files from dirs in the data dir
files = glob.glob('data/*/*.*')

In [62]:
# prepare dataset in an array
dataset = []

# loop through every file that "glob" found.
for filepath in files:
	# regex used for Windows/MacOS compatibility
	filecrawl = re.split(r'\\+|/+', filepath)

	# remove the "data" folder entry; its not needed
	filecrawl = filecrawl[1:]

	# tag images from the fire-images folder as "fire"
	if filecrawl[0] == 'Fire images':
		filecrawl.append(1)
	else:
		filecrawl.append(0)

	# add filecrawl findings to dataset
	dataset.append(filecrawl)
	# ==NOTE==
	# Because the "glob" package arbitrarily crawls files, the
	# index of an item may not be the same each time this is run.

### Create Dataframe

In [63]:
# create our project's main dataframe
dataframe = pandas.DataFrame(dataset, columns=['folder', 'filename', 'fire'])

# display shows these tables neatly, shown below
display(dataframe.head(), dataframe.tail())

,folder,filename,fire
0,Normal Images 3,Hotel_Monterey_La_Soeur_Osaka_standard_twin_be...,0
1,Normal Images 3,house5.jpg,0
2,Normal Images 3,mi-plage-hawai.jpg,0
3,Normal Images 3,JB224_03_Colourful_christmas_table_setting_in_...,0
4,Normal Images 3,Interior Design Ideas (3).jpg,0


,folder,filename,fire
646,Normal Images 1,amazing-modern-living-room-design-with-white-b...,0
647,Normal Images 1,813e6b10-848b-478e-a471-480f14eeb271.1.10.jpg,0
648,Normal Images 1,201504-connect-norway-horizontal-1b.jpg,0
649,Normal Images 1,690x380-Lake-Tahoe-Summer-Emerald-Bay.jpg,0
650,Normal Images 1,AAEAAQAAAAAAAAJqAAAAJDAwNDUwNDAwLWUwYTAtNDlmNy...,0


## Step 2: Data Manipulation

### Check for Duplicates

In [64]:
# check for any duplicate filenames in the dataframe
duplicates = dataframe['filename'].duplicated(keep=False)

# Select rows with duplicate filenames
duplicate_rows = dataframe[duplicates]
display(duplicate_rows)

# create a warning if duplicates exist
if duplicates.sum() != 0:
	warning = Warning(
		f'There are {duplicates.sum()}'
		' duplicated filenames in the dataframe.'
		' proceed with caution.'
	) 
	display(warning)

,folder,filename,fire
24,Normal Images 3,maxresdefault.jpg,0
68,Normal Images 3,images.jpg,0
415,Fire images,14.jpg,1
419,Fire images,maxresdefault.jpg,1
490,Fire images,1.jpg,1
491,Fire images,images.jpg,1
536,Normal Images 1,14.jpg,0
644,Normal Images 1,1.jpg,0


Warning('There are 8 duplicated filenames in the dataframe. proceed with caution.')

### Rebalance Datapoints
I used [this article][rebalancing] to help figure things out.

[rebalancing]: https://towardsdatascience.com/having-an-imbalanced-dataset-here-is-how-you-can-solve-it-1640568947eb

In [65]:
# == FIXME ==
# this does not seem to be working properly at the moment

"""
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.tree import DecisionTreeClassifier

# create an object of the classifier, called "rebalancinator"
rebalancinator = BalancedBaggingClassifier(
	base_estimator=DecisionTreeClassifier(),
	sampling_strategy='auto',
	replacement=False,
	random_state=0
)

'''
# train the classifier.
rebalancinator.fit(x_train, y_train)
preds = rebalancinator.predict(x_train)
'''
"""

pass

### Train/Test Split
*Why do we need train/test split?*
*What does it do?*

In [66]:
# for x: keep the dataframe but drop the fire column
x = dataframe.drop(columns=['fire'])

# for y: drop everything in the dataframe but fire
y = dataframe.loc[:, ['fire']]

In [67]:
# Use the built-in "train test split" function
# to generate the four desireable segments of data.
x_train, x_test, y_train, y_test = train_test_split(
	x, y, test_size=0.35)

x_train = x_train.reset_index().drop('index', 1)
y_train = y_train.reset_index().drop('index', 1)
x_test = x_test.reset_index().drop('index', 1)
y_test = y_test.reset_index().drop('index', 1)

display(x_test.head())
display(y_test.head())

,folder,filename
0,Normal Images 2,attachment-image-1cfe3cc1-a2b6-402c-a50e-8436e...
1,Normal Images 2,buntzen-lake-full-width-nature.jpg
2,Fire images,girl-on-fire-01.jpg
3,Normal Images 5,wallpapers_1.jpg
4,Normal Images 3,khon_kaen_photo_36.jpg


,fire
0,0
1,0
2,1
3,0
4,0


### Create Image Vectors

In [68]:
image_length = 256

In [69]:
def get_img_vector(x, index):
	# find filepath
	filename = x['filename'][index]
	folder = x['folder'][index]
	filepath = f'data/{folder}/{filename}'

	# open the image via its filepath
	img = Image.open(filepath)
	# ==NOTE==
	# the Image class was imported from PIL
	# (python image library)

	# remove transparency layer
	img = img.convert('RGB')

	# resize the image
	img = img.resize((image_length, image_length))

	# return the image vector
	return img_to_array(img)

In [70]:
def data_gen(x, y, batch_size):
	# n_batch variables are empty arrays of constant size.
	# x_batch is has RGB values for each pixel's coordinate.
	# y_batch represents whether there is fire or not (0/1).
	x_batch = numpy.zeros((batch_size, image_length, image_length, 3))
	y_batch = numpy.zeros((batch_size, 1))

	# loop through entire dataframe, index by index
	for index in range(len(x)):
		# using batch_size, we can determine 
		x_batch[index % batch_size] = get_img_vector(x, index)
		y_batch[index % batch_size] = y['fire'][index]

		# if there has been {batch_size} items, we yield.
		# the last batch is an outlier; its batch is smaller.
		if ((index + 1) % batch_size == 0
		or (index + 1) == len(dataframe)):
			yield (x_batch, y_batch)

## Step 3: Train Model

In [71]:
# use "sequential" mode from keras module
# see https://jovianlin.io/keras-models-sequential-vs-functional/
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# model = Sequential()
# model.add(Conv2D(16, (4,4), activation='relu', input_shape=(image_length, image_length, 3)))
# model.add(Flatten())
# model.add(Dense(128, activation='relu'))
# model.add(Dense(16, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))

In [72]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(image_length, image_length, 3)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [73]:
model.compile(
	loss=binary_crossentropy,
	optimizer=Adadelta(),
	metrics=['accuracy']
)

In [74]:
batch_size = 64
epochs = 5

for epoch in range(epochs):
	print(f"Epoch {epoch+1} / {epochs}")
	for x_batch, y_batch in data_gen(x_train, y_train, batch_size):
		model.train_on_batch(
			x_batch, y_batch
		)
		loss, accuracy = model.evaluate(x_batch, y_batch)
		print('accuracy:', accuracy)

Epoch 1 / 5
64/64 [==============================] - 3s 53ms/step
accuracy: 0.890625
64/64 [==============================] - 3s 46ms/step
accuracy: 0.828125
64/64 [==============================] - 3s 46ms/step
accuracy: 0.84375
64/64 [==============================] - 3s 45ms/step
accuracy: 0.828125
64/64 [==============================] - 3s 46ms/step
accuracy: 0.875
64/64 [==============================] - 3s 45ms/step
accuracy: 0.75
Epoch 2 / 5
64/64 [==============================] - 3s 45ms/step
accuracy: 0.890625
64/64 [==============================] - 3s 45ms/step
accuracy: 0.828125
64/64 [==============================] - 3s 45ms/step
accuracy: 0.84375
64/64 [==============================] - 3s 46ms/step
accuracy: 0.828125
64/64 [==============================] - 3s 45ms/step
accuracy: 0.875
64/64 [==============================] - 3s 46ms/step
accuracy: 0.75
Epoch 3 / 5
64/64 [==============================] - 5s 74ms/step
accuracy: 0.890625
64/64 [========================

## Step 4: Affirm Model Accuracy

In [85]:
def get_y_value(index):
	return y_test['fire'][index]

def get_y_guess(index):
	fire = model.predict(get_img_vector(x_test, index).reshape(-1, image_length, image_length, 3))
	if fire > 0.3:
		return 1
	else:
		return 0

def get_confusion_matrix_objects():
	y_values = []
	y_guesses = []
	for index in list(range(len(x_test))):
		y_values.append(get_y_value(index))
		y_guesses.append(get_y_guess(index))
	return (y_values, y_guesses)

In [87]:
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(*get_confusion_matrix_objects()).ravel()

In [89]:
print(
	f' true positive {tp}\n'
	f' true negative {tn}\n'
	f'false positive {fp}\n'
	f'false negative {fn}\n'
)

 true positive 0
 true negative 189
false positive 0
false negative 39



In [77]:
for index in list(range(50)):
	predict = model.predict(get_img_vector(x_test, index).reshape(-1, image_length, image_length, 3))
	print(
		f'\n{predict[0][0]} \n' 
		'<img src="./data/'
		f'{x_train["folder"][index]}'
		'/'
		f'{x_train["filename"][index]}">'
	)


0.0 
<img src="./data/Normal Images 5/ReineFishingVillage.jpg">

0.0 
<img src="./data/Normal Images 1/45d893bb4a9e15e3ea7aab81ec783e7c.jpg">

0.0 
<img src="./data/Fire images/images.jpg">

0.0 
<img src="./data/Normal Images 3/mda.jpg">

0.0 
<img src="./data/Normal Images 5/Standard-single-1600x900.jpg">

0.0 
<img src="./data/Normal Images 3/image3tffh.jpg">

0.0 
<img src="./data/Fire images/pic1.jpg">

0.0 
<img src="./data/Normal Images 2/grimstad gate 02.jpg">

0.0 
<img src="./data/Normal Images 2/b4d7b6e907c47ebb74058bac9d1953f1d138cb43b660420e4e42037915b11720.jpg">

0.0 
<img src="./data/Normal Images 5/uncle-clay-s-house-of.jpg">

0.0 
<img src="./data/Normal Images 1/14566688_115393625608382_6451053875163037696_n.jpg">

0.0 
<img src="./data/Normal Images 5/room6_450_reload.jpg">

0.0 
<img src="./data/Normal Images 2/b33375c5ff1a2f222ebdca2b0046eecc.jpg">

0.0 
<img src="./data/Normal Images 1/500_F_128074665_71JRncSY29Pdumr6kt8sPLYvByWxeWXc.jpg">

0.0 
<img src="./data/

<img src="./data/Fire images/images.jpg">
<img src="./data/Fire images/11_10_19-mjs_ft_hotel-fire_19183862.jpg">
<img src="./data/Fire images/FlamesEverett House Fire.transfer_1461845684828_4041471_ver1.0_640_360.jpg">
